In [1]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from pyspark import SparkContext, SQLContext
from pyspark.mllib.evaluation import MulticlassMetrics,BinaryClassificationMetrics
import numpy as np

In [2]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(np.float64(values[0]), np.float64(values[1:]))

In [3]:
sc = SparkContext(appName="LogisticRegressionWithLBFGS Classification")

In [4]:
data = sc.textFile("C:\Users\Rachan\Downloads\Assignment 1\Assignment 1\winequality-white_Set1.csv")
parsedData = data.map(parsePoint)


In [5]:
print parsedData.first()

(1.0,[7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8])


In [6]:
# 60,40 split traing, tesing data
trainingData, testData = parsedData.randomSplit([3, 2], 11L)

In [7]:
print ("Training Data Count : "+str(trainingData.count()))
print ("Test Data Count : "+str(testData.count()))

Training Data Count : 2958
Test Data Count : 1940


In [15]:
# Build the Training model 
# Parameter variations
trainingModel = LogisticRegressionWithLBFGS.train(trainingData, iterations=200, tolerance=0.001, regType='l1', intercept=True)

AttributeError: 'NoneType' object has no attribute 'sc'

In [9]:
trainingPredsAndLabels = trainingData.map(lambda p: (float(trainingModel.predict(p.features)),float(p.label)))

In [10]:
trainErr = trainingPredsAndLabels.filter(lambda (v, p): v != p).count() / float(trainingData.count())
print("Training Error = " + str(trainErr))
metrics = MulticlassMetrics(trainingPredsAndLabels)
binarymetrics = BinaryClassificationMetrics(trainingPredsAndLabels)
print("Training Precision = " + str(metrics.precision()))
print("Training Precision(1) = " + str(metrics.precision(1)))
print("Training Precision(0) = " + str(metrics.precision(0)))
print("Training Recall = " + str(metrics.recall()))
print("Training Recall(1) = " + str(metrics.recall(1)))
print("Training Recall(0) = " + str(metrics.recall(0)))
print("Training Area Under ROC = " + str(binarymetrics.areaUnderROC))


Training Error = 0.216700473293
Training Precision = 0.783299526707
Training Precision(1) = 0.625
Training Precision(0) = 0.787248787249
Training Recall = 0.783299526707
Training Recall(1) = 0.0682852807284
Training Recall(0) = 0.988255763375
Training Area Under ROC = 0.528270522052


In [11]:
testPredsAndLabels = testData.map(lambda p: (float(trainingModel.predict(p.features)),float(p.label)))

In [12]:
# Testing data metrics
testErr = testPredsAndLabels.filter(lambda (v, p): v != p).count() / float(testData.count())
print("Testing Error = " + str(testErr))
metrics = MulticlassMetrics(testPredsAndLabels)
binarymetrics = BinaryClassificationMetrics(testPredsAndLabels)
print("Training Precision = " + str(metrics.precision()))
print("Training Precision(1) = " + str(metrics.precision(1)))
print("Training Precision(0) = " + str(metrics.precision(0)))
print("Training Recall = " + str(metrics.recall()))
print("Training Recall(1) = " + str(metrics.recall(1)))
print("Training Recall(0) = " + str(metrics.recall(0)))
print("Training Area Under ROC = " + str(binarymetrics.areaUnderROC))

Testing Error = 0.196907216495
Training Precision = 0.803092783505
Training Precision(1) = 0.75
Training Precision(0) = 0.804210526316
Training Recall = 0.803092783505
Training Recall(1) = 0.0746268656716
Training Recall(0) = 0.993498049415
Training Area Under ROC = 0.534062457543


In [13]:
sc.stop()